# Web scrapper

In [1]:
from datetime import datetime
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import os
os.environ['CLASSPATH'] = "./KotlinInside-1.10.2-all.jar"

from jnius import autoclass

gall = 'kizunaai' #갤러리 id
start_year=datetime.now().year
end_year=datetime.now().year

user=autoclass("be.zvz.kotlininside.session.user.Anonymous")
inside=autoclass("be.zvz.kotlininside.KotlinInside")
http=autoclass("be.zvz.kotlininside.http.DefaultHttpClient")

test=inside.createInstance(user("ㅇㅇ","zhxmfflsakstp"), http(True,True))
test=inside(user("ㅇㅇ","zhxmfflsakstp"), http(True,True), True)
test.generateClientToken()
# db 디렉토리를 생성, 초기화
# for year in range(start_year,end_year+1): #2018~2019년 폴더 생성

#     for month in range(1,12+1): #1~12월 폴더 생성
#         baseloc = os.path.abspath('./%s/%s/%s/' % (gall,year,month)) # 경로
        
#         if not os.path.exists(baseloc+'table'): os.makedirs(baseloc+'table')
#         if not os.path.exists(baseloc+'word'):os.makedirs(baseloc+'word') #table, word 폴더는 다른 통계 코드용
#         pd.DataFrame(columns=['번호','제목','날짜','닉네임','ID/IP','조회 수','달린 댓글 수','추천 수','비추 수','content','mobile','개념글 수','idtype']).to_json(baseloc+'post.json')
#         pd.DataFrame(columns=['번호','날짜','닉네임','ID/IP','dccon','content','idtype','답글 대상','댓삭 당한 횟수']).to_json(baseloc+'comment.json')
# RDQ1L1NtWGFWR3RDenZuNzhBMlBBbWJNbzllYmgwaVZZNFB6djNJUlFGWT0=
# RDQ1L1NtWGFWR3RDenZuNzhBMlBBZy9DQnpmUG9BR0pTTnNINnk4ekVrMD0=

'UFUzUnI4N1NKOVdTd3gyeVU3UzBQRDQ4RmVRTWhhNzBKV3gvRXh5OUExTT0='

In [2]:
print(test.generateAppId())

UFUzUnI4N1NKOVdTd3gyeVU3UzBQQjBVZkZzcEZqMTlzNjkxNXNSOHd2TT0=


In [4]:
import gevent.monkey
gevent.monkey.patch_socket()
gevent.monkey.patch_ssl()
import re,requests,sys,time,html,random
from gevent.pool import Pool
import numpy as np
import pandas as pd
import string
import html
from base64 import b64encode
from hashlib import sha256

session = requests.Session()
app_id=0

def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

def get_app_id(): #공앱에서 사용하는 app_id 값 받아오기. 주기적으로 새로 발급해야함
    app_id = test.generateAppId()
    return app_id

def gethtml(no): #글 수집
    count = 0 #에러 체크용 카운트, 10 넘으면 스크랩 패스
    url = "http://m.dcinside.com/api/gall_view_new.php?id="+gall+"&no="+str(no)+"&app_id="+app_id
    while True:
        try:
            a = session.get('http://m.dcinside.com/api/redirect.php?hash='+b64encode(url.encode('utf-8')).decode('utf-8'),headers={"User-Agent": "dcinside.app"},timeout=5)
            if r'\uae00\uc5c6\uc74c' in a.text: return; # 삭제된 글 체크
            intro = a.json(strict=False)[0]['view_info'];view = a.json(strict=False)[0]['view_main']
            postcontent = view['memo'] #글 내용
            postIP = intro['user_id'] if len(intro['ip'])==0 else intro['ip'] #작성자 IP
            mobile = False if intro['write_type']=='W' else True #모바일 체크
            recom = False if intro['recommend_chk'] == 'N' else True #개념글 체크
            break
        except requests.exceptions.RequestException as e:pass
        except requests.Timeout as e:pass
        except Exception as e:
            if not('<!DOCTYPE html>' in a.text or a.text == ''):print(no,'idf', end=' ');print(str(e));print(a.text)
            count+=1
            if count > 10:sys.exit()
    
    #댓글
    replypage = 1;total_comment = 0;passnick = 'deleted';passIP = 'deleted'
    while True:
        mobileurl = "http://m.dcinside.com/api/redirect.php?hash="+b64encode(("http://m.dcinside.com/api/comment_new.php?csort=new&id="+gall+"&no="+str(no)+"&re_page="+str(replypage)+"&app_id="+app_id+"=").encode('utf-8')).decode('utf-8')+"%3D%3D"
        count=0 #에러 체크용 카운트, 10 넘으면 스크랩 패스
        while True:
            try:
                a = session.get(mobileurl,headers={"User-Agent": "dcinside.app"},timeout=5)
                comment = a.json(strict=False)[0]
                for comm in comment['comment_list']:
                    if not('ipData' in comm.keys()): comm['ipData'] = ''
                    if 'under_step' in comm.keys(): target = '%s (%s)' % (passnick,passIP);
                    else: passnick = comm['name']; passIP = comm['user_id'] if len(comm['ipData'])==0 else comm['ipData']; target = None
                    IP = comm['user_id'] if len(comm['ipData'])==0 else comm['ipData']
                    dccon = True if 'dccon' in comm.keys() else False
                    if 'is_delete_flag' in comm.keys():
                        if '작성자' in comm['is_delete_flag']: removed_by_writer = True
                        else: removed_by_writer = False
                    else: removed_by_writer = False
                    content = comm['comment_memo'] if dccon==False else comm['dccon']
                    commlist.append({u'번호':no,u'날짜':comm['date_time'],u'닉네임':comm['name'],'ID/IP':IP,'idtype':comm['member_icon'],'content':content, 'dccon':dccon,
                                     '답글 대상':target, '댓삭 당한 횟수':removed_by_writer})
                break
            except requests.exceptions.RequestException as e:pass
            except requests.Timeout as e:pass
            except Exception as e:
                if not('<!DOCTYPE html>' in a.text or a.text == ''): print(no,'cdf', end=' ');print(str(e)); print(a.text)
                count+=1
                if count > 5:sys.exit()

        if len(comment['comment_list']) == 0: break
        else: total_comment+= len(comment['comment_list'])
        replypage+=1
    postlist.append({u'번호':no,u'제목':intro['subject'],u'날짜':intro['date_time'],u'닉네임':intro['name'],'ID/IP':postIP,'idtype':intro['member_icon'],
                     u'조회 수':intro['hit'],u'달린 댓글 수':total_comment,u'추천 수':view['recommend'],u'비추 수':view['nonrecommend'],
                     'content':postcontent,'mobile':mobile,u'개념글 수':recom})


In [ ]:
from datetime import datetime
start = 2499139 #시작 번호 (젤 최근 게시글 번호)
end = 2185219 # 종료 번호
gall = 'kizunaai';year=datetime.now().year; month =datetime.now().month-1 # 갤러리, 연, 월
baseloc = os.path.abspath('./%s/%s/%s/' % (gall,year,month)) # 경로
turn = 400 # step 한국어로 뭐라하는지 모름

pool = Pool()
print("Ready")
idf = pd.read_json(baseloc+'post.json');cdf = pd.read_json(baseloc+'comment.json')
app_id=get_app_id()
exit=False
total_val = (start-end+1)//turn+1
with tqdm(total=total_val) as pbar: # 소요 시간 출력
    while True:
        start_time = time.perf_counter() # 시간 측정용 현재 시간 저장
        commlist = [];postlist = [] # dataframe 변환용 list 초기화
        pl = list(range(start+1-turn,start+1)); pl.reverse()
#         postlist = idf[u'번호'].values.tolist() # 중복 글 체크용 번호 목록 생성
        for postnum in pl:
            if postnum < end: 
                exit = True
                break # 종료 번호보다 오래된 글 일시 패스
#             if postnum in postlist: 
#                 continue # 중복 글 체크용인데 보통은 안 쓰니 주석처리 해놓는게 처리속도 더 빠름
            pool.spawn(gethtml,postnum)
        pool.join()
        print(len(postlist), end = " |") # 수집 글 갯수
        pbar.update(1) # 소요 시간 업데이트
        start = start - turn
        if len(postlist) != 0: pd.concat([idf, pd.DataFrame(postlist)],sort=False).reset_index(drop=True).to_json(baseloc+'post.json')
        if len(commlist) != 0: pd.concat([cdf, pd.DataFrame(commlist)],sort=False).reset_index(drop=True).to_json(baseloc+'comment.json') #저장
        if exit==True: break
        if len(postlist) != 0: app_id=get_app_id();idf = pd.read_json(baseloc+'post.json');cdf = pd.read_json(baseloc+'comment.json') # db에 추가된 거 있을시 다시 로드
        

Ready


  0%|          | 0/785 [00:00<?, ?it/s]

395 |392 |389 |389 |385 |388 |381 |379 |387 |380 |382 |365 |378 |383 |384 |386 |395 |384 |370 |355 |362 |373 |379 |386 |387 |385 |394 |391 |398 |383 |385 |387 |387 |378 |376 |377 |383 |386 |393 |376 |361 |352 |377 |379 |390 |389 |383 |383 |381 |385 |380 |382 |383 |373 |383 |380 |380 |380 |372 |386 |378 |366 |381 |367 |393 |392 |395 |387 |390 |372 |386 |373 |375 |392 |390 |380 |385 |378 |395 |365 |380 |388 |381 |376 |388 |372 |382 |373 |386 |382 |386 |388 |380 |392 |376 |381 |365 |384 |364 |383 |383 |391 |393 |387 |389 |388 |388 |388 |387 |382 |382 |386 |390 |385 |385 |384 |384 |381 |386 |381 |372 |369 |363 |374 |359 |383 |383 |383 |380 |376 |358 |371 |371 |361 |382 |383 |384 |380 |379 |384 |375 |380 |386 |382 |386 |382 |367 |356 |359 |376 |377 |377 |379 |351 |386 |374 |388 |386 |373 |378 |377 |380 |382 |365 |370 |365 |347 |367 |386 |395 |383 |383 |389 |381 |364 |370 |374 |384 |374 |373 |369 |367 |389 |382 |382 |380 |380 |375 |387 |370 |373 |380 |389 |381 |380 |372 |375 |377 |373 |374 |

In [ ]:
# 중복글 제거
# idf = pd.read_json(baseloc+'post.json');
# cdf = pd.read_json(baseloc+'comment.json') # db에 추가된 거 있을시 다시 로드

# idf.drop_duplicates(['번호']).to_json(baseloc+'post.json')
# cdf.drop_duplicates(['번호']).to_json(baseloc+'comment.json') #저장


In [ ]:
# html 제거
idf = pd.read_json(baseloc+'post.json');
cdf = pd.read_json(baseloc+'comment.json') # db에 추가된 거 있을시 다시 로드

# cdf.drop_duplicates(['번호']).to_json(baseloc+'comment.json') #저장
idf["content"]=idf["content"].str.replace(r'<[^<]+?>','',regex=True)
idf.to_json(baseloc+'post.json')


# tagging

In [1]:
from PyKomoran import *

import os
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from datetime import datetime, timedelta
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
import jpype,time,json,sys,calendar,json,collections
from datetime import datetime
from tqdm.notebook import tqdm
import vtuber_dict as vd

vtuber_dict=vd.vtuber
nicknames = vd.nicknames
dict_file = '/mnt/e/helloworld/user_dict.txt'
userdict_df=pd.read_csv(dict_file, names=["단어"], index_col=None)

In [2]:
# 자동 사전 단어 추가
for vtuber, nicknames in tqdm(vtuber_dict.items()):
    for nickname in nicknames:
        if not userdict_df["단어"].isin([nickname]).any():
            to_append = [nickname]
            userdict_df.loc[len(userdict_df)] = to_append
userdict_df.to_csv(dict_file, header=False, index=False)

  0%|          | 0/325 [00:00<?, ?it/s]

In [3]:
gall = 'kizunaai';year=datetime.now().year; month =datetime.now().month-1 # 갤러리, 연, 월
baseloc = os.path.abspath('./%s/%s/%s/' % (gall,year,month)) # 경로

one_word_filter = userdict_df['단어'][userdict_df['단어'].str.len() == 1].tolist()
count_tags=defaultdict(int)

def get_kodict():
    ko=Komoran("STABLE")
    ko.set_user_dic(dict_file)
    return ko

try:
    jpype.startJVM()
except:
    pass

k = get_kodict()
arr_filter=[]


def tagging(tags, count_tags, a):
    try:
        jpype.java.lang.Thread.attach()
    except:
        pass
    
#         if len(a) > 1000:continue
    if not arr_filter and any(word in a for word in arr_filter):
        return list()
    try:
        nouns = k.nouns("\n".join([s for s in a.split("\n") if s]));nouns = list(set(nouns)); count = Counter(nouns)
    except:
        return

    count_list = list(count)
    for word in count_list:
        if not word in one_word_filter and len(word) == 1:
            del count[word]
            
    for n in list(count.elements()):
#         if not n in tags.keys():
#             tags[n]=1
#         else:
        tags[n]+=1
          
        if n in nicknames:
            for vtuber, calls in vtuber_dict.items():
                if n in calls:
#                     if not vtuber in count_tags.keys():
#                         count_tags[vtuber]=1
#                     else:
                    count_tags[vtuber]+=1
                    break
    return

In [4]:
idf = pd.read_json(baseloc+'post.json');cdf = pd.read_json(baseloc+'comment.json')
cdf=cdf[cdf['dccon']==False]
cont = pd.concat([idf, cdf],sort=False)
cont[u'날짜'] = pd.to_datetime(cont[u'날짜'],format='%Y.%m.%d')
cont["content"]=cont["content"].str.replace(r'<[^<]+?>','',regex=True)
first = str(year)+'-'+str(month)
last = str(year)+'-'+str(month+1) if month != 12 else str(year+1)+'-1'


for beg in tqdm(pd.date_range(first, last,closed='left', freq='D'), position=0):
    acont = cont[(cont[u'날짜'] >= beg + timedelta(hours=6)) & (cont[u'날짜'] < beg + timedelta(hours=30))]
    start_time = time.time()
    print (beg.month,beg.day,end=' '),
    econt = acont[(acont[u'날짜'] >= beg + timedelta(hours=6)) & (acont[u'날짜'] < beg + timedelta(hours=18))]
    alines = econt[u'제목']+'\n'+econt['content']
    alines = alines.dropna().tolist()
    lines =alines
    tags = defaultdict(int) 
    print('Number of lines in document:',end=' '),
    nlines = len(lines)
    print (nlines,end=' ')
    for line in lines:
        a = str(line)
        tagging(tags, count_tags, a)
#     t1 = Process(target=do_concurrent_tagging, args=(0, int(nlines/2), lines))
#     t2 = Process(target=do_concurrent_tagging, args=(int(nlines/2), nlines, lines))
#     t1.start(); t2.start(); t1.join(); t2.join()
    print("%.2f" % (time.time() - start_time) +u'초',end=' ')
    os.remove(baseloc+'word/'+str(beg.day)+'_day.json')
    with open(baseloc+'word/'+str(beg.day)+'_day.json', 'w') as f:
        json.dump(tags.copy(), f)

    start_time = time.time()
    tags = defaultdict(int)
    econt = acont[(acont[u'날짜'] < beg + timedelta(hours=30)) & (acont[u'날짜'] >= beg + timedelta(hours=18))]
    alines = econt[u'제목']+'\n'+econt['content']
    alines = alines.dropna().tolist()
    lines =alines
    
    print('Number of lines in document:',end=' '),
    nlines = len(lines)
    print (nlines,end=' ')
#     with Pool(3) as p:
    for line in lines:
        a = str(line)
        tagging(tags, count_tags, a)
#     t1 = Process(target=do_concurrent_tagging, args=(0, int(nlines/2), lines))
#     t2 = Process(target=do_concurrent_tagging, args=(int(nlines/2), nlines, lines))
#     t1.start(); t2.start();t1.join(); t2.join()
    print("%.2f" % (time.time() - start_time) +u'초')
    #print ' '.join(tags)
    os.remove(baseloc+'word/'+str(beg.day)+'_night.json')
    with open(baseloc+'word/'+str(beg.day)+'_night.json', 'w') as f:
        json.dump(tags.copy(), f)

totaltag = []
lastday = calendar.monthrange(year,month)[1]
for beg in range(1,lastday+1):
    with open(baseloc+'word/'+str(beg)+'_day.json', 'r') as f:totaltag.append(dict(json.load(f)))
    with open(baseloc+'word/'+str(beg)+'_night.json', 'r') as f:totaltag.append(dict(json.load(f)))
counter = collections.Counter()
for d in totaltag:counter.update(d)
tagsdict = dict(counter)

  0%|          | 0/31 [00:00<?, ?it/s]

8 1 Number of lines in document: 3237 8.26초 Number of lines in document: 8038 20.25초
8 2 Number of lines in document: 2022 5.87초 Number of lines in document: 3763 8.33초
8 3 Number of lines in document: 2207 6.21초 Number of lines in document: 4181 10.41초
8 4 Number of lines in document: 1825 4.40초 Number of lines in document: 3490 7.57초
8 5 Number of lines in document: 2046 4.93초 Number of lines in document: 4012 8.73초
8 6 Number of lines in document: 1743 4.20초 Number of lines in document: 4347 9.46초
8 7 Number of lines in document: 2481 5.27초 Number of lines in document: 5469 12.10초
8 8 Number of lines in document: 2586 5.68초 Number of lines in document: 7432 15.38초
8 9 Number of lines in document: 2719 6.12초 Number of lines in document: 5698 12.80초
8 10 Number of lines in document: 2344 5.73초 Number of lines in document: 5852 13.65초
8 11 Number of lines in document: 1948 4.57초 Number of lines in document: 7072 14.12초
8 12 Number of lines in document: 2190 5.02초 Number of lines in doc

# Wordcloud

In [5]:
import matplotlib.pyplot as plt
os.makedirs(baseloc, exist_ok=True)
cur_dir='./'
r=["!!!",".com",".kr",".co",".be","오늘","진짜","이거","사람","시간","생각","시작","하나","해서","이상","들이","하면","얘기","하게","하네","어제","뭔가","안보","그거","고싶","그새","이번"]
for k in r:tagsdict.pop(k, None)
maskfile = np.array(Image.open(os.path.abspath(f"{cur_dir}/ai.jpg")))
wc = WordCloud(font_path=r'/mnt/c/Windows/Fonts/BMDOHYEON_ttf.ttf',
               mode = "RGBA", background_color=None,
               max_words=1000,
               mask=maskfile,
               prefer_horizontal=1,
               min_font_size=0,
               max_font_size=150,
               width=1398.425197, height=2116.535433, scale=5
                ).generate_from_frequencies(tagsdict)
image_colors = ImageColorGenerator(maskfile)
wc.recolor(color_func=image_colors, random_state=0).to_file(f"{baseloc}/wordcloud.png")

In [6]:
from parsers import parser
import vtuber_dict as vd
import os
from datetime import datetime
gall = 'kizunaai';year=datetime.now().year; month =datetime.now().month-1 # 갤러리, 연, 월
baseloc = os.path.abspath('./%s/%s/%s/' % (gall,year,month)) # 경로

parser(baseloc, vd.nijisanji, 'sum_niji.csv')
parser(baseloc, vd.hololive, 'sum_holo.csv')
parser(baseloc, vd.vtuber, 'sum_ai.csv')
parser(baseloc, vd.exceptholo, 'sum_noholo.csv')

2번째 바 길이: 16.892876427829695
3번째 바 길이: 13.057528556593976
4번째 바 길이: 9.742554517133955
5번째 바 길이: 9.520913811007267
6번째 바 길이: 8.827082035306333
7번째 바 길이: 7.323779854620975
8번째 바 길이: 6.919044652128763
9번째 바 길이: 5.666292834890965
10번째 바 길이: 5.425379023883696
Top10에서 사라진:      Vtuber                                           단어(언급 수)  총 언급 횟수
14   셀렌 타츠키                                     셀렌(491)+타츠키(1)      492
16   군도 미레이  군도(343)+미레이(85)+도 미레이(43)+미래이(2)+미.레이(0)+미1레이(...      473
17   마치타 치마                                            치마(472)      472
24  유우키 치히로          치히로(185)+치재앙(157)+마법소녀(28)+치여사(13)+치쨩(12)      395
2번째 바 길이: 10.775435083608478
3번째 바 길이: 8.761094898285227
4번째 바 길이: 8.729466396847375
5번째 바 길이: 7.571072531686015
6번째 바 길이: 7.377347960379167
7번째 바 길이: 6.837686654595803
8번째 바 길이: 6.596519331132176
9번째 바 길이: 6.37314303972734
10번째 바 길이: 6.337560975609756
Top10에서 사라진:       Vtuber                                           단어(언급 수)  총 언급 횟수
11   아마네 카나타  가짜(1451)+가짜가짜(744)+

In [6]:
import vtuber_dict as vd
import word_dict as wd
from tqdm.notebook import tqdm
from datetime import datetime
import os
import pandas as pd

# df[df['col'].str.contains("hello|train",na=False)]

first = str(year)+'-'+str(month)
last = str(year)+'-'+str(month+1) if month != 12 else str(year+1)+'-1'
vtuber_dict=vd.vtuber
word_dict=wd.word_dict
total_col=['Vtuber','관련 글/댓글 수']

for award in word_dict:
    total_col.append(award)
    


In [7]:
try:
    cont["제목내용"]=cont[u'제목'].astype(str)+'\n'+cont['content'].astype(str)
except NameError as e:
    idf = pd.read_json(baseloc+'post.json');cdf = pd.read_json(baseloc+'comment.json')
    cdf=cdf[cdf['dccon']==False]
    cont = pd.concat([idf, cdf],sort=False)
    cont["제목내용"]=cont[u'제목'].astype(str)+'\n'+cont['content'].astype(str)
    
cont["제목내용"]=cont["제목내용"].str.replace(r'<[^<]+?>','',regex=True)
total_info=[]
total_percent_info=[]

reader_path=baseloc+'/sum_ai.csv'
df = pd.read_csv(reader_path, index_col=0)
target_vtuber=df["Vtuber"].values
count_tags_keys = count_tags.keys()
for vtuber in tqdm(target_vtuber):
    award_col=[vtuber]
    award_percent_col=[vtuber]
    fillters=''
    
    for nickname in vtuber_dict[vtuber]:
        fillters+= nickname+'|'
    fillters=fillters[:-1]
#     print(fillters)
    vdtuer_df=cont[cont['제목내용'].str.contains(fillters,na=False, regex = True)][['제목내용']]
#     print(vdtuer_df['제목내용'])
    
    vtuber_count = count_tags[vtuber] if vtuber in count_tags_keys else 0
    award_col.append(vtuber_count)
    award_percent_col.append(vtuber_count)
    for award in word_dict:
        award_fillters=''
        for word in word_dict[award]:
            award_fillters+= word+'|'
        award_fillters=award_fillters[:-1]
#         print(award_fillters)
        award_df=vdtuer_df[vdtuer_df['제목내용'].str.contains(award_fillters,na=False, regex = True)]
        award_count=len(award_df)
#         print(award_df['제목내용'])
        if award_count == 0 or vtuber_count == 0:
            award_percent_col.append(0)
        else:
            award_percent_col.append(award_count/vtuber_count*100)
        award_col.append(award_df['제목내용'].count())
#     print(award_fillters)
    
    total_info.append(award_col)
    total_percent_info.append(award_percent_col)
#     break
    
total_per_df = pd.DataFrame(total_percent_info, columns=total_col)
total_per_df.to_csv(baseloc+'/award_percent.csv', sep=',', index=True)

total_df = pd.DataFrame(total_info, columns=total_col)
total_df.to_csv(baseloc+'/award.csv', sep=',', index=True)

  0%|          | 0/291 [00:00<?, ?it/s]

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm.notebook import tqdm
import os 

old_month = month - 1
old_year = year
if month is 1:
    old_month = 12
    old_year = year - 1
oldloc=os.path.abspath('./%s/%s/%s/' % (gall,old_year,old_month))


# pd.read_csv(baseloc+"/award.csv")
# for award in word_dict:
#     print(f"{award} Award")
#     print(total_df.sort_values(by=[award], ascending=False).head(n=1))
    
# pd.read_csv(baseloc+"/award_percent.csv")
for award in tqdm(word_dict):
    print(f"{award} Award")
    total_per_df50 = total_per_df.sort_values(by=["관련 글/댓글 수"], ascending=False).head(n=50)
    print(total_per_df50.sort_values(by=[award], ascending=False).head(n=3))
    print("\n")

  0%|          | 0/10 [00:00<?, ?it/s]

게이머 Award
       Vtuber  관련 글/댓글 수        게이머        성희롱         나쁜말          중계  \
38  니노마에 이나니스        848  85.259434  55.542453  135.849057  280.778302   
44    유우키 치히로        678  47.935103   8.849558   26.991150  112.831858   
46     셀렌 타츠키        636  46.540881   6.289308   35.220126   71.226415   

          호감       비호감        미인       못생김         꿀잼         노잼  
38  3.419811  0.471698  5.306604  0.589623  18.160377  10.023585  
44  0.737463  0.294985  0.442478  0.000000   2.949853   0.442478  
46  7.547170  0.628931  2.830189  1.572327   3.301887   1.415094  


성희롱 Award
       Vtuber  관련 글/댓글 수        게이머         성희롱         나쁜말          중계  \
18    나키리 아야메       2214  12.646793  145.076784   38.211382   68.654020   
0      사쿠라 미코       5438  11.456418   79.753586   25.947039  100.478117   
38  니노마에 이나니스        848  85.259434   55.542453  135.849057  280.778302   

          호감       비호감        미인       못생김         꿀잼         노잼  
18  0.858175  0.496838  1.851852  0.225836   

In [9]:
output_filename='commentpost_rank.csv'
total_commentpost_df=total_per_df.sort_values(by=["관련 글/댓글 수"], ascending=False)[["Vtuber","관련 글/댓글 수"]]

total_commentpost_df.index = np.arange(1, len(total_commentpost_df) + 1)

try:
    old_data = pd.read_csv(os.path.join(oldloc, output_filename))
except:
    old_data = pd.DataFrame()
compared_ranks = []
for idx, member in tqdm(total_commentpost_df.iterrows(), total=len(total_commentpost_df)):
    vtuber = member["Vtuber"]
    try:
        old_rank = old_data[old_data["Vtuber"] == vtuber].index[0]+1
    except:
        old_rank = "NA"

    if old_rank == "NA":
        compared_rank = f"NEW"
    elif idx > old_rank:
        compared_rank = f"▼{idx-old_rank}"
    elif idx < old_rank:
        compared_rank = f"▲{old_rank-idx}"
    elif idx == old_rank:
        compared_rank = f"■-"
    else:
        compared_rank="Error"
    compared_ranks.append(compared_rank)

total_commentpost_df["전월 대비 순위"]=compared_ranks
total_commentpost_df.to_csv(baseloc+'/commentpost_rank.csv', sep=',', index=True)

print(total_commentpost_df.head(n=10))


  0%|          | 0/291 [00:00<?, ?it/s]

       Vtuber  관련 글/댓글 수 전월 대비 순위
1     아마네 카나타       5439      ▲32
2      사쿠라 미코       5438       ■-
3     우사다 페코라       5139       ■-
4       호쇼 마린       4026       ▲1
5     토코야미 토와       3946       ▲5
6     아카이 하아토       3339      ▲25
7    츠노마키 와타메       3230       ▲4
8     아토리 코토코       3146      NEW
9   호시마치 스이세이       3087       ▼1
10   나츠이로 마츠리       3017      ▲19


In [11]:
print(sum(total_commentpost_df["관련 글/댓글 수"]))

116355


# Youtube

In [1]:
import pandas as pd
import re
import urllib.parse as urlparse
import requests
from datetime import datetime
import os
from tqdm.notebook import tqdm
import ray
ray.init()
from gevent.pool import Pool

# https://stackoverflow.com/questions/4356538/how-can-i-extract-video-id-from-youtubes-link-in-python
def video_id_parse(value):
    """
    Examples:
    - http://youtu.be/SA2iWivDJiE
    - http://www.youtube.com/watch?v=_oPAwA_Udwc&feature=feedu
    - http://www.youtube.com/embed/SA2iWivDJiE
    - http://www.youtube.com/v/SA2iWivDJiE?version=3&amp;hl=en_US
    """
    
    query = urlparse.urlparse(value)
    if query.hostname == 'youtu.be':
        return query.path[1:]
    if query.hostname in ('www.youtube.com', 'youtube.com'):
        if query.path == '/watch':
            p = urlparse.parse_qs(query.query)
            return p['v'][0]
        if query.path[:7] == '/embed/':
            return query.path.split('/')[2]
        if query.path[:3] == '/v/':
            return query.path.split('/')[2]
    # fail?
    return None

gall = 'kizunaai';year=datetime.now().year; month =datetime.now().month-1 # 갤러리, 연, 월
baseloc = os.path.abspath('./%s/%s/%s/' % (gall,year,month)) # 경로
  # https://stackoverflow.com/questions/28292822/how-to-know-uploader-of-a-video-using-youtube-v3-api
youtube_query='https://www.googleapis.com/youtube/v3/videos?fields=items(snippet(channelTitle))&part=snippet&id={YOUR_VIDEO_ID}&key={API_KEY}'
# https://stackoverflow.com/questions/4705996/python-regex-convert-youtube-url-to-youtube-video/19161373
# Regex
yt_regex = r'(https?://)?(www\.)?(youtube|youtu|youtube-nocookie)\.(com|be)/(watch\?v=|embed/|v/|.+\?v=)?([^&=%\?]{11})'

columns=["ChannelName", "VideoID", "Count"]


2021-09-02 08:54:06,944	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8265


In [5]:

try:
    cont["제목내용"]=cont[u'제목'].astype(str)+'\n'+cont['content'].astype(str)
    yout_cont = cont[cont['제목내용'].str.contains(yt_regex, na=False, regex = True)][['제목내용']]
except NameError as e:
    idf = pd.read_json(baseloc+'post.json');cdf = pd.read_json(baseloc+'comment.json')
    cdf=cdf[cdf['dccon']==False]
    cont = pd.concat([idf, cdf],sort=False)
    cont["제목내용"]=cont[u'제목'].astype(str)+'\n'+cont['content'].astype(str)
    yout_cont = cont[cont['제목내용'].str.contains("(youtube)|(youtu.be)", na=False, regex = True)][['제목내용']]
if os.path.isfile(baseloc+'/youtube.csv'):
    yt_df=pd.read_csv(baseloc+'/youtube.csv',index_col=None)
    yt_df["VideoID"]=yt_df["VideoID"].str.replace(r"[가-힣|ㄱ-ㅎ|ㅏ-ㅣ ]+","",regex=True).str.replace(r"(https://www.youtube.com/watch)|(https://m.youtube.com/watch)","",regex=True)
    yt_df["Count"]=0
else:
    yt_df=pd.DataFrame(columns=columns)

yout_cont["제목내용"]=yout_cont["제목내용"].str.replace(r'<[^<]+?>','',regex=True)
req_count=0
NUM=0

yt_regex_id = ray.put(yt_regex)
@ray.remote
def parse_contents(contents, yt_regex):
    processed_video_ids = []
    for content in contents:
        content=str(content)
        video_ids = [x.group() for x in re.finditer(yt_regex, content)]
        
        for raw_video_id in video_ids:
            video_id = str(video_id_parse(str(raw_video_id)))
            if video_id is None:
                continue
            video_id = video_id.replace("https://www.youtube.com/watch","").replace("https://m.youtube.com/watch","")
            video_id = re.sub(r"[가-힣|ㄱ-ㅎ|ㅏ-ㅣ ]+","",video_id)
            processed_video_ids.append([video_id])
            
    return processed_video_ids

result_ids=[]
contents = []
batch_size = 1000
for idx, row in tqdm(yout_cont.iterrows(), total=len(yout_cont)):
    content = row["제목내용"]
    contents.append(content)
    if len(contents) >= batch_size:
        result_ids.append(parse_contents.remote(contents, yt_regex_id))
        contents=[]
    
with tqdm(total=len(result_ids)) as pbar: # 소요 시간 출력
    yt_arr = []
    while len(result_ids):
        done_id, result_ids = ray.wait(result_ids)
        test=ray.get(done_id)

        video_ids = test[0]
        yt_arr.extend(video_ids)
        pbar.update(1)
        
raw_yt_df=pd.DataFrame(yt_arr, columns=["VideoID"])
raw_yt_df = raw_yt_df[raw_yt_df["VideoID"] != "None"]
yt_df=raw_yt_df.value_counts().to_frame(name="Count")

yt_df.sort_values(by=['Count'], ascending=False, inplace=True)
yt_df.to_csv(baseloc+'/youtube.csv', sep=',')

<ipython-input-5-c346f438c2ad>:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  yout_cont = cont[cont['제목내용'].str.contains(yt_regex, na=False, regex = True)][['제목내용']]


  0%|          | 0/187071 [00:00<?, ?it/s]

  0%|          | 0/187 [00:00<?, ?it/s]

TypeError: Index(...) must be called with a collection of some kind, 'VideoID' was passed

In [ ]:
yt_df

In [2]:
# Unknown 재탐색
yt_df=pd.read_csv(baseloc+'/youtube.csv',index_col=None)
KEYS=["PUT_YOUR_API_KEY"]

@ray.remote
def getChannelName(idx, video_id, key):
    try:
        youtube_info = requests.get(youtube_query.format(YOUR_VIDEO_ID=video_id, API_KEY=key)).json()
        channel_name = youtube_info['items'][0]['snippet']['channelTitle']
    except:
        channel_name = "Unknown"
    return idx, channel_name
    
req_count = 0
NUM = 0
result_ids = []
for idx, row in tqdm(yt_df.iterrows(), total=len(yt_df)):
    channel_name=str(row["ChannelName"])
    if channel_name == "Unknown":
        video_id = str(row["VideoID"])
        if req_count > 9000:
            NUM+=1
            req_count=0
        req_count+=1

        result_ids.append(getChannelName.remote(idx, video_id, KEYS[idx%len(KEYS)]))


while len(result_ids):
    done_id, result_ids = ray.wait(result_ids)
    test=ray.get(done_id)
#     print(test)
# out: [(31, 'Unknown')]

    idx, channel_name = test[0]
    yt_df.loc[idx,'ChannelName'] = channel_name

yt_df.to_csv(baseloc+'/youtube.csv',index=False, sep=',')

  0%|          | 0/11375 [00:00<?, ?it/s]

[(31, 'Unknown')]


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
print(yt_df.head(n=10))

In [ ]:
print(f"total number of post: {idf['번호'].count()}")
print(idf.sort_values(by=['추천 수'], ascending=False).head(n=1))
print("\n")
print(idf.sort_values(by=['조회 수'], ascending=False).head(n=1))
print("\n")
print(idf.sort_values(by=['달린 댓글 수'], ascending=False).head(n=1))
print("\n")
print(idf["개념글 수"].value_counts())